<a href="https://colab.research.google.com/github/shackste/galaxy-generator/blob/separate_py_files/ACGAN_CVAE_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PyTorch implementation of combined Conditional Variable Auto Encoder (CVAE) and Auxiliary-Classifier Generative Adversarial Network (ACGAN)

continuation of work by Mohamad Dia

# Environment Setup

## Modules

In [1]:
!pip install torchviz

     |████████████████████████████████| 51kB 7.4MB/s 
  Created wheel for torchviz: filename=torchviz-0.0.1-cp37-none-any.whl size=3520 sha256=1f7dd6ea90d371e94cbfe5ae5b60f89324c60608d2bd0c219356122aec960574
  Stored in directory: /root/.cache/pip/wheels/2a/c2/c5/b8b4d0f7992c735f6db5bfa3c5f354cf36502037ca2b585667
Successfully built torchviz


In [2]:


import sys 

import matplotlib.pyplot as plt
from torch import cat, add, ones, zeros


## include separate module files
!git clone -b separate_py_files https://github.com/shackste/galaxy-generator.git

sys.path.insert(0,"/content/galaxy-generator/python_modules/")

from parameter import labels_dim, input_size, parameter
from file_system import root, folder_results
from helpful_functions import summarize, write_generated_galaxy_images_iteration
from sampler import make_training_sample_generator
from dataset import get_x_train, get_labels_train
from loss import loss_discriminator, loss_generator

import pipeline
from pipeline import VAE, VAEGAN
from discriminator import Discriminator4
from encoder import Encoder4
from decoder import Decoder4


Cloning into 'galaxy-generator'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 162 (delta 103), reused 98 (delta 47), pack-reused 0
Receiving objects: 100% (162/162), 3.16 MiB | 30.54 MiB/s, done.
Resolving deltas: 100% (103/103), done.

!!!!!!!!!!

galaxyzoo_data_cropped_nonnormalized.npy and training_solutions_rev1.csv must be placed in google drive under FHNW/galaxy_generator/
the results will be placed there, too.

Mounted at /drive


# basic test Neural Networks
check whether the pipeline components work on their own

In [3]:
summarize(Discriminator4)

input size (3, 64, 64)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 64, 64]              64
         LeakyReLU-2           [-1, 16, 64, 64]               0
            Conv2d-3           [-1, 32, 32, 32]           4,640
       BatchNorm2d-4           [-1, 32, 32, 32]              64
         LeakyReLU-5           [-1, 32, 32, 32]               0
            Conv2d-6           [-1, 64, 16, 16]          18,496
       BatchNorm2d-7           [-1, 64, 16, 16]             128
         LeakyReLU-8           [-1, 64, 16, 16]               0
            Conv2d-9            [-1, 128, 8, 8]          73,856
      BatchNorm2d-10            [-1, 128, 8, 8]             256
        LeakyReLU-11            [-1, 128, 8, 8]               0
          Flatten-12                 [-1, 8192]               0
           Linear-13                 [-1, 1024]       8,389,632
        LeakyReL

In [4]:
import numpy as np
from torch import rand


net = Encoder4()

print(np.sum([np.prod(p.size()) for p in net.parameters()]) == 8508656)
input_dummy = rand(3, *input_size)
label_dummy = rand(3, labels_dim)

net(input_dummy, label_dummy)

True


(tensor([[ 0.1421, -0.2423,  0.0967,  0.3992, -0.2161,  0.3253,  0.1492, -0.2746],
         [-0.3237, -0.1038, -0.5041, -0.3167, -0.3121, -0.7072, -0.0351, -0.2488],
         [ 0.2505, -0.0255, -0.2822, -0.2313, -0.4929, -0.0486, -0.2691,  0.4607]],
        grad_fn=<AddmmBackward>),
 tensor([[0.4920, 0.7047, 0.7827, 0.4335, 1.0000, 0.7958, 0.9132, 0.7743],
         [1.0283, 0.6942, 0.4761, 0.4233, 1.2211, 1.0086, 1.2905, 0.5837],
         [0.5365, 0.7953, 0.6508, 0.5631, 0.3876, 0.7153, 0.7852, 1.0525]],
        grad_fn=<SoftplusBackward>))

In [5]:
net = Decoder4()

print(np.sum([np.prod(p.size()) for p in net.parameters()]))
input_dummy = rand(3, parameter.latent_dim)
label_dummy = rand(3, labels_dim)

net(input_dummy, label_dummy).shape

8524675


torch.Size([3, 3, 64, 64])

In [6]:
net = VAE()
images_dummy = rand(5,3,64,64).cuda()
labels_dummy = rand(5,3).cuda()
pred = net(images_dummy, labels_dummy)
pred.shape

torch.Size([5, 3, 64, 64])

In [7]:
net = VAEGAN()
input_dummy = rand(3,3,64,64).cuda()
labels_dummy = rand(3,3).cuda()
pred = net(input_dummy, labels_dummy)
pred.shape, pred[:,0]

(torch.Size([3, 1028]),
 tensor([0.5037, 0.5509, 0.5390], device='cuda:0', grad_fn=<SelectBackward>))

# Training Data
load data from files at google drive

In [8]:
x_train = get_x_train()
labels_train = get_labels_train()
N_samples = x_train.shape[0]

# Training

In [9]:
epochs = 20
batch_size = 128
steps = N_samples // batch_size
save_interval = 200

discriminator_losses = []
discriminator_losses_real = []
discriminator_losses_fake = []
generator_losses = []

valid = ones((batch_size,1)).cuda()
fake = zeros((batch_size,1)).cuda()

## if you want to change discriminator, encoder or decoder network:
#pipeline.decoder = YourDecoder().cuda()
## before you create the VAE and VAEGAN

## if you want to change any parameter:
#parameter.alpha = 1
## this can be done during runtime

vae = VAE()
vaegan = VAEGAN()


iteration = 0
epoch = 0
step = 0



In [10]:
def training_step():
    global iteration
    images, labels = next(sample_generator)

    # -------------------
    # Train Discriminator
    # -------------------
    vae.train(False)
    pipeline.discriminator.train(True)
    pipeline.discriminator.zero_grad()

    generated_images = vae(images, labels)
    target_real = cat((valid,labels), dim=1)
    prediction_real = pipeline.discriminator(images)[:,:1+labels_dim]
    target_fake = cat((fake, labels), dim=1)
    prediction_fake = pipeline.discriminator(generated_images)[:,:1+labels_dim]

    d_loss_real = loss_discriminator(target_real, prediction_real)
    d_loss_fake = loss_discriminator(target_fake, prediction_fake)
    d_loss = 0.5 * add(d_loss_fake, d_loss_real)
    discriminator_losses.append(d_loss)
    discriminator_losses_fake.append(d_loss_fake)
    discriminator_losses_real.append(d_loss_real)

    d_loss_real.backward()
    d_loss_fake.backward()
    pipeline.discriminator.optimizer.step()

    # ---------------
    # Train Generator
    # ---------------
    vae.train(True)
    pipeline.discriminator.train(False)
    pipeline.encoder.zero_grad()
    pipeline.decoder.zero_grad()

    generated_images = vae(images, labels)
    target = pipeline.discriminator(images)
    target[:,0] = 1
    target[:,1:1+labels_dim] = labels
    target = target.detach()
    prediction = pipeline.discriminator(generated_images)
    latent = pipeline.encoder(images, labels)

    g_loss = loss_generator(target, prediction, images, generated_images, latent)
    g_loss.backward()
    pipeline.encoder.optimizer.step()
    pipeline.decoder.optimizer.step()
    generator_losses.append(g_loss)


    iteration += 1

    print(f"iteration {iteration}, epoch {epoch+1}, batch {step+1}/{steps}," + \
          f"disc_loss {d_loss:.5}, (real {d_loss_real:.5}, fake {d_loss_fake:.5} ) gen_loss {g_loss:.5}")

    if not iteration % save_interval:
        write_generated_galaxy_images_iteration(iteration=iteration, images=generated_images.detach().cpu().numpy())


In [ ]:
while epoch < epochs:
    sample_generator = make_training_sample_generator(batch_size, x_train, labels_train)
    step = 0
    while step < steps:
        training_step()
        step += 1
    epoch += 1

    # save a plot of the costs
    plt.clf()
    plt.plot(discriminator_losses, label='discriminator cost')
    plt.plot(generator_losses, label='generator cost')
    plt.plot(discriminator_losses_fake, label='discriminator cost fake', linestyle=":")
    plt.plot(discriminator_losses_real, label='discriminator cost real', linestyle="-.")
    plt.yscale("log")
    plt.legend()
    plt.savefig(folder_results+"cost_vs_iteration.png")
    plt.close()


    ### really save?
    pipeline.decoder.save()
    pipeline.encoder.save()
    pipeline.discriminator.save()

iteration 1, epoch 1, batch 1/481,disc_loss 142.28, (real 138.34, fake 146.23 ) gen_loss 772.25
iteration 2, epoch 1, batch 2/481,disc_loss 135.46, (real 130.95, fake 139.96 ) gen_loss 2221.5
iteration 3, epoch 1, batch 3/481,disc_loss 247.76, (real 279.3, fake 216.22 ) gen_loss 1509.5
iteration 4, epoch 1, batch 4/481,disc_loss 128.71, (real 126.66, fake 130.77 ) gen_loss 1184.3
iteration 5, epoch 1, batch 5/481,disc_loss 120.39, (real 125.02, fake 115.76 ) gen_loss 1026.8
iteration 6, epoch 1, batch 6/481,disc_loss 101.81, (real 105.59, fake 98.025 ) gen_loss 875.14
iteration 7, epoch 1, batch 7/481,disc_loss 91.274, (real 96.622, fake 85.927 ) gen_loss 827.23
iteration 8, epoch 1, batch 8/481,disc_loss 103.93, (real 111.41, fake 96.455 ) gen_loss 703.81
iteration 9, epoch 1, batch 9/481,disc_loss 91.88, (real 98.718, fake 85.043 ) gen_loss 683.74
iteration 10, epoch 1, batch 10/481,disc_loss 94.382, (real 101.65, fake 87.112 ) gen_loss 646.13
iteration 11, epoch 1, batch 11/481,disc

iteration 200, epoch 1, batch 200/481,disc_loss 82.981, (real 89.79, fake 76.173 ) gen_loss 307.26
iteration 201, epoch 1, batch 201/481,disc_loss 85.92, (real 91.435, fake 80.406 ) gen_loss 315.42
iteration 202, epoch 1, batch 202/481,disc_loss 86.646, (real 91.206, fake 82.086 ) gen_loss 327.78
iteration 203, epoch 1, batch 203/481,disc_loss 81.776, (real 89.709, fake 73.842 ) gen_loss 317.79
iteration 204, epoch 1, batch 204/481,disc_loss 82.505, (real 89.883, fake 75.127 ) gen_loss 336.92
iteration 205, epoch 1, batch 205/481,disc_loss 77.415, (real 84.123, fake 70.707 ) gen_loss 315.64
iteration 206, epoch 1, batch 206/481,disc_loss 81.903, (real 87.929, fake 75.877 ) gen_loss 337.74
iteration 207, epoch 1, batch 207/481,disc_loss 82.124, (real 86.978, fake 77.269 ) gen_loss 343.68
iteration 208, epoch 1, batch 208/481,disc_loss 87.515, (real 95.323, fake 79.706 ) gen_loss 336.55
iteration 209, epoch 1, batch 209/481,disc_loss 83.095, (real 87.984, fake 78.206 ) gen_loss 319.51
it

# Testing

In [ ]:
!pip install line_profiler
%load_ext line_profiler
%lprun -f training_step training_step()

In [ ]:
labels_train.dtype